In [1]:
from bombergym.scenarios import classic_with_opponents
from bombergym.environments import register
import gym

In [2]:
register()
settings, agents = classic_with_opponents()
env = gym.make('BomberGym-v4', args=settings, agents=agents)

In [44]:
dict = generate_expert_traj(expert, save_path="test-experttraj", env=env, n_timesteps=10000, n_episodes=100)

state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
state is none
actions (22173, 1)
obs (22173, 5, 17, 17)
rewards (22173,)
episode_returns (100,)
episode_

In [45]:
dict


KeyboardInterrupt



In [3]:
import logging
from bombergym.agent_code.rule_based_agent.callbacks import act, setup
from bombergym.settings import ACTIONS

class Self:
    logger = logging.getLogger("Self")

self=Self()
setup(self)

In [4]:
def expert(obs, env, other):
    if other is None:
        state = env.env.initial_state
    else:
        state = other["orig_state"]
    if state is None:
        print(f'state is none')
        return ACTIONS.index("WAIT")
    action = act(self, state)
    if action is None:
        return ACTIONS.index("WAIT")
    return ACTIONS.index(action)

In [3]:
# Adapted for Bombergym from
# https://github.com/Stable-Baselines-Team/stable-baselines/blob/master/stable_baselines/gail/dataset/record_expert.py
import os
import warnings
from typing import Dict

import cv2  # pytype:disable=import-error
import numpy as np
from gym import spaces

from stable_baselines3.common.vec_env import VecEnv, VecFrameStack

def generate_expert_traj(model, save_path=None, env=None, n_timesteps=0,
                         n_episodes=100, image_folder='recorded_images'):
    """
    Train expert controller (if needed) and record expert trajectories.
    .. note::
        only Box and Discrete spaces are supported for now.
    :param model: (RL model or callable) The expert model, if it needs to be trained,
        then you need to pass ``n_timesteps > 0``.
    :param save_path: (str) Path without the extension where the expert dataset will be saved
        (ex: 'expert_cartpole' -> creates 'expert_cartpole.npz').
        If not specified, it will not save, and just return the generated expert trajectories.
        This parameter must be specified for image-based environments.
    :param env: (gym.Env) The environment, if not defined then it tries to use the model
        environment.
    :param n_timesteps: (int) Number of training timesteps
    :param n_episodes: (int) Number of trajectories (episodes) to record
    :param image_folder: (str) When using images, folder that will be used to record images.
    :return: (dict) the generated expert trajectories.
    """
    assert env is not None, "You must set the env in the model or pass it to the function."

    assert (isinstance(env.observation_space, spaces.Box) or
            isinstance(env.observation_space, spaces.Discrete)), "Observation space type not supported"

    assert (isinstance(env.action_space, spaces.Box) or
            isinstance(env.action_space, spaces.Discrete)), "Action space type not supported"

    actions = []
    observations = []
    rewards = []
    episode_returns = np.zeros((n_episodes,))
    episode_starts = []

    ep_idx = 0
    obs = env.reset()
    episode_starts.append(True)
    reward_sum = 0.0
    idx = 0
    # state and mask for recurrent policies
    state, mask = None, None
    other=None

    while ep_idx < n_episodes:
        observations.append(obs)
        action = model(obs, env, other)

        obs, reward, done, other = env.step(action)

        actions.append(action)
        rewards.append(reward)
        episode_starts.append(done)
        reward_sum += reward
        idx += 1
        if done:
            obs = env.reset()
            # Reset the state in case of a recurrent policy
            state = None

            episode_returns[ep_idx] = reward_sum
            reward_sum = 0.0
            ep_idx += 1

    observations = np.concatenate(observations).reshape((-1,) + env.observation_space.shape)
    #elif isinstance(env.observation_space, spaces.Discrete):
    #    observations = np.array(observations).reshape((-1, 1))
    #elif record_images:
    #    observations = np.array(observations)

    if isinstance(env.action_space, spaces.Box):
        actions = np.concatenate(actions).reshape((-1,) + env.action_space.shape)
    elif isinstance(env.action_space, spaces.Discrete):
        actions = np.array(actions).reshape((-1, 1))

    rewards = np.array(rewards)
    episode_starts = np.array(episode_starts[:-1])

    assert len(observations) == len(actions)

    numpy_dict = {
        'actions': actions,
        'obs': observations,
        'rewards': rewards,
        'episode_returns': episode_returns,
        'episode_starts': episode_starts
    }  # type: Dict[str, np.ndarray]

    for key, val in numpy_dict.items():
        print(key, val.shape)

    if save_path is not None:
        np.savez(save_path, **numpy_dict)

    env.close()

    return numpy_dict

ModuleNotFoundError: No module named 'stable_baselines3'

In [3]:
from stable_baselines import PPO2
from stable_baselines.gail import ExpertDataset
# Using only one expert trajectory
# you can specify `traj_limitation=-1` for using the whole dataset
dataset = ExpertDataset(expert_path='test-experttraj.npz',
                        traj_limitation=1, batch_size=128)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



No protocol specified
/home/g556b/.pyenv/versions/3.7.12/envs/bomberman_rl_exp_oldpy/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


actions (22173, 1)
obs (22173, 5, 17, 17)
rewards (22173,)
episode_returns (100,)
episode_starts (22173,)
Total trajectories: 1
Total transitions: 465
Average returns: 3.16
Std for returns: 2.6710297639674483


In [4]:

model = PPO2('MlpPolicy', env, verbose=1)
# Pretrain the PPO2 model
model.pretrain(dataset, n_epochs=1000)

# As an option, you can train the RL agent
# model.learn(int(1e5))

# Test the pre-trained model
env = model.get_env()
obs = env.reset()

reward_sum = 0.0
for _ in range(1000):
        action, _ = model.predict(obs)
        obs, reward, done, _ = env.step(action)
        reward_sum += reward
        env.render()
        if done:
                print(reward_sum)
                reward_sum = 0.0
                obs = env.reset()

env.close()


Wrapping the env in a DummyVecEnv.




Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Pretraining with Behavior Cloning...


ValueError: Cannot feed value of shape (128, 1445) for Tensor 'input/Ob:0', which has shape '(?, 5, 17, 17)'